# Déployer sur un point de terminaison de lot

Imaginez qu’une policlinique relève les mesures de patients toute la journée, en enregistrant les détails de chaque patient dans un fichier séparé. Ensuite, pendant la nuit, le modèle de prédiction du diabète peut être utilisé pour traiter toutes les données patient de la journée en tant que lot, en générant des prédictions qui attendront le lendemain matin afin que la clinique puisse suivre les patients qui sont prédits comme étant à risque de diabète. Avec Azure Machine Learning, vous pouvez faire cela en créant un point de terminaison de lot, et c’est ce que vous allez implémenter dans cet exercice.

## Avant de commencer

Vous devez avoir la dernière version du package **azureml-ai-ml** pour exécuter le code dans ce notebook. Exécutez la cellule ci-dessous pour vérifier qu’il est installé.

> **Remarque** :
> Si le package **azure-ai-ml** n’est pas installé, exécutez `pip install azure-ai-ml` pour l’installer.

In [ ]:
## Se connecter à un espace de travail

Une fois les packages requis du Kit de développement logiciel (SDK) installés, vous êtes prêt à vous connecter à votre espace de travail.

Pour vous connecter à un espace de travail, vous avez besoin de paramètres d’identificateur : un ID d’abonnement, un nom de groupe de ressources et un nom d’espace de travail. Le nom du groupe de ressources et le nom de l’espace de travail sont déjà renseignés pour vous. Vous avez seulement besoin de l’ID d’abonnement pour exécuter la commande.

Pour trouver les paramètres nécessaires, cliquez sur l’abonnement et le nom de l’espace de travail en haut à droite du studio. Un volet s’ouvre à droite.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiez l’ID d’abonnement et remplacez **YOUR-SUBSCRIPTION-ID** par la valeur que vous avez copiée. </p>

## Inscrire le modèle

Les déploiements par lots peuvent uniquement déployer des modèles inscrits dans l’espace de travail. Vous allez inscrire un modèle MLflow, qui est stocké dans le dossier `model` local. 

In [ ]:
## Créer un point de terminaison de traitement de lots

Un point de terminaison de lot est un point de terminaison HTTPS que les applications peuvent appeler pour déclencher un travail de scoring par lots. Le nom d’un point de terminaison de lot doit être unique dans une région Azure. Vous allez utiliser la fonction `datetime` pour générer un nom unique basé sur la date et l’heure actuelles. 

In [ ]:
Pour créer un point de terminaison avec la classe `BatchEndpoint`, vous devez spécifier le nom et éventuellement une description. Après avoir créé un point de terminaison, vous allez y déployer un modèle.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT ! Attendez que le point de terminaison soit créé avant de continuer ! Une notification verte devrait apparaître dans le studio. </p>

## Créer le déploiement

Un déploiement est un ensemble de ressources nécessaires pour héberger le modèle qui effectue l’inférence réelle. Nous allons créer un déploiement pour notre point de terminaison en utilisant la classe `BatchDeployment`. 

Étant donné que vous déployez un modèle MLflow, vous n’avez pas besoin d’un script de scoring ni de définir l’environnement. Azure Machine Learning crée automatiquement ces éléments pour vous. Le fichier `MLmodel` dans le dossier `model` est utilisé pour comprendre quelles sont les entrées et sorties attendues du modèle.

Vous allez déployer un modèle avec les paramètres suivants :

- `name` : nom du déploiement.
- `description` : description facultative pour clarifier davantage ce que représente le déploiement.
- `endpoint_name` : nom du point de terminaison créé précédemment sur lequel le modèle devrait être déployé.
- `model` : nom du modèle inscrit.
- `compute` : calcul à utiliser lors de l’appel du modèle déployé pour générer des prédictions.
- `instance_count` : nombre de nœuds de calcul à utiliser pour générer des prédictions.
- `max_concurrency_per_instance` : nombre maximal d’exécutions du script de scoring parallèles par nœud de calcul.
- `mini_batch_size` : nombre de fichiers par exécution du script de scoring.
- `output_action` : chaque nouvelle prédiction est ajoutée en tant que nouvelle ligne au fichier de sortie.
- `output_file_name` : fichier auquel les prédictions sont ajoutées.
- `retry_settings` : paramètres si un mini-lot échoue.
- `logging_level` : niveau de verbosité du journal. Les valeurs autorisées sont `warning`, `info` et `debug`. 

L’exécution de la cellule suivante configure et crée le déploiement.

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT ! Attendez que le déploiement soit terminé avant de poursuivre ! Une notification verte devrait apparaître dans le studio. </p>

Vous pouvez déployer plusieurs modèles sur un point de terminaison. Vous pouvez définir le déploiement par défaut pour spécifier le modèle à utiliser par défaut lors de l’appel d’un point de terminaison de lot.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT ! Attendez que le déploiement par défaut soit défini avant de poursuivre ! Une notification verte devrait apparaître dans le studio. </p>

## Préparer les données pour les prédictions par lots

Dans le dossier `data`, vous trouverez des fichiers CSV avec des données sans étiquette. Vous allez créer une ressource de données qui pointe vers les fichiers du dossier `data`, que vous utiliserez comme entrée pour le travail par lots.

In [ ]:
## Envoi du travail

Maintenant que vous avez déployé un modèle sur un point de terminaison de lot et que vous disposez d’une ressource de données sans étiquette, vous êtes prêt à appeler le point de terminaison pour générer des prédictions sur les données sans étiquette.

Tout d’abord, vous allez définir l’entrée en référençant la ressource de données inscrite. Ensuite, vous allez appeler le point de terminaison, qui soumettra un travail de pipeline. Vous pouvez utiliser l’URL du travail pour le surveiller dans le studio. Le travail contient un travail enfant qui représente l’exécution du script de scoring (généré) pour obtenir les prédictions.

## Obtenir les résultats

Lorsque le travail de pipeline qui appelle le point de terminaison de lot est terminé, vous pouvez voir les résultats. Toutes les prédictions sont collectées dans le fichier `predictions.csv` qui est stocké dans le magasin de données par défaut. Vous pouvez télécharger le fichier et visualiser les données en exécutant les cellules suivantes. 

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

In [ ]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score